In [8]:
import logging as log
import yaml

In [10]:
def run_contract_scan(datacontract_yml_path: str, file_path: str):
    with open(datacontract_yml_path, "r") as infile:
        checks_filepath = yaml.safe_load(infile.read())["archivo_de_revision_de_datos"]
        with open(checks_filepath, "r") as checks_file:
            checks = checks_file.read()
    # log.info(checks)
    print(checks)

    scan = Scan()
    scan.set_scan_definition_name("test")
    scan.set_data_source_name("dask")

    ddf = dd.read_csv('data/emoji_usage_dataset.csv')

    scan.add_dask_dataframe(dataset_name="emoji", dask_df=ddf)

run_contract_scan(datacontract_yml_path = "contracts/data_contract_emoji.yml")

checks for emoji:
    - row_count > 0
    - invalid_count("User Age") = 0:
        valid min: 26
        valid max: 52
    - avg("User Age") > 37
    - duplicate_count(Emoji) = 0
    - schema: # Checks for schema changes
        name: Find forbidden, missing, or wrong type
        warn:
            when required column missing: [Emoji, "User Gender"]
            when forbidden column present: [credit_card]
            when wrong column type:
                Platform: varchar
        fail:
            when forbidden column present: [pii*]
            when wrong column index:
                "User Age": 1

